<a href="https://colab.research.google.com/github/jmoussa99/DS-HW/blob/main/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## [Jamal Moussa, XRZ2515]

# Obtain the data

In [ ]:
#Run this code block, Don't change anything
from sklearn import datasets
from sklearn.model_selection import train_test_split
cal = datasets.fetch_california_housing()
X = cal.data
y = cal.target
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)
y_bool_train = [i > 1.8 for i in y_train]
y_bool_test = [i > 1.8 for i in y_test]

# Q1

### Q1a.

In [ ]:
#TODO
def Q1o():
  pass

In [ ]:
gbr_model=Q1o()

In [ ]:
#TODO
def Q1a():
  pass

In [ ]:
Q1a()

### Q1b.

#### plot

In [ ]:
#TODO
def Q1b():
  pass

In [ ]:
Q1b()

#### conclusion

In [ ]:
#TODO
def Q1b_finding():
  finding=""
  return finding

In [ ]:
Q1b_finding()

### Q1c.

In [ ]:
#TODO
def Q1c_finding():
  finding=""
  return finding

In [ ]:
Q1c_finding()

# Q2.

### Q2a.

In [ ]:
#TODO
def Q2a():
  pass

In [ ]:
Q2a()

### Q2b.

In [ ]:
#TODO
def Q2bo():
  pass

In [ ]:
dt_model=Q2bo()

In [ ]:
#TODO
def Q2b():
  pass

In [ ]:
Q2b()

# Q3.

### Q3a.

In [ ]:
#TODO
def Q3a():
  pass

In [ ]:
Q3a()

### Q3b.

In [ ]:
#TODO
def Q3b():
  pass

In [ ]:
Q3b()

### Q3c.

In [ ]:
#TODO
def Q3c():
  pass

In [ ]:
Q3c()

### Q3d.

In [ ]:
#TODO
def Q3d():
  pass

In [ ]:
Q3d()